In [ ]:
from farm_haystack.reader.adaptive_model import FARMReader
from farm_haystack.retriever.tfidf import TfidfRetriever
from farm_haystack import Finder
from farm_haystack.indexing.io import write_documents_to_db, fetch_archive_from_http
from farm_haystack.indexing.cleaning import clean_wiki_text
from farm_haystack.utils import print_answers

## Task: Question Answering for Game of Thrones

<img style="float: right;" src="https://upload.wikimedia.org/wikipedia/en/d/d8/Game_of_Thrones_title_card.jpg">

Question Answering can be used in a variety of use cases. A very common one:  Using it to navigate through complex knowledge bases or long documents ("search setting").

A "knowledge base" could for example be your website, an internal wiki or a collection of financial reports. 
In this tutorial we will work on a slightly different domain: "Game of Thrones". 

Let's see how we can use a bunch of wikipedia articles to answer a variety of questions about the 
marvellous seven kingdoms...  


## Indexing & cleaning documents

In [ ]:
# Init a database (default: sqllite)
from farm_haystack.database import db
db.create_all()

# Let's first get some documents that we want to query
# Here: 517 Wikipedia articles for Game of Thrones
doc_dir = "data/article_txt_got"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# Now, let's write the docs to our DB. 
# You can supply a cleaning/transformation function that is applied to each doc (e.g. to remove headers)
# It must take a str as input, and return a str.
# Our 517 documents also get splitted into 2068 smaller paragraphs here. 
write_documents_to_db(document_dir=doc_dir, clean_func=clean_wiki_text)

## Initalize Reader, Retriever & Finder

In [ ]:
# A retriever identifies the k most promising chunks of text that might contain the answer for our question
# Retrievers use some simple but fast algorithm, here: TF-IDF
retriever = TfidfRetriever()

In [ ]:
# A reader scans the text chunks in detail and extracts the k best answers
# Reader use more powerful but slower deep learning models, here: a BERT QA model trained via FARM on Squad 2.0
reader = FARMReader(model_dir="../FARM/saved_models/bert-english-qa-large")

In [ ]:
# The Finder sticks together retriever and retriever in a pipeline to answer our actual questions 
finder = Finder(reader, retriever)

## Voilá! Ask a question!

In [ ]:
prediction = finder.get_answers(question="Who is the father of Arya Stark?", top_k_reader=10, top_k_retriever=5)

In [ ]:
#prediction = finder.get_answers(question="Who created the Dothraki vocabulary?", top_k_reader=5)
#prediction = finder.get_answers(question="Who is the sister of Sansa?", top_k_reader=5)

In [ ]:
print_answers(prediction, details="minimal")